In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='5'
import torch

if torch.cuda.is_available():
    print("✅ CUDA is available!")
    print("GPU count:", torch.cuda.device_count())
    print("Current GPU:", torch.cuda.get_device_name(0))
else:
    print("❌ No GPU detected.")


✅ CUDA is available!
GPU count: 1
Current GPU: Tesla V100-SXM2-32GB


In [3]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# === DATA PATHS ===
data_dir = "/home/23ucs747/mini-project-sem5/dataset_split"

# === TRANSFORMS ===
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# === LOAD DATASETS ===
train_data = datasets.ImageFolder(root=f"{data_dir}/train", transform=train_transforms)
test_data = datasets.ImageFolder(root=f"{data_dir}/test", transform=test_transforms)

# === DATALOADERS ===
train_loader = DataLoader(train_data, batch_size=16, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False, num_workers=4)

# === CHECK ===
print(f"Train: {len(train_data)} images")
print(f"Test: {len(test_data)} images")
print(f"Classes: {train_data.classes}")


Train: 3348 images
Test: 840 images
Classes: ['Blight', 'Common_Rust', 'Gray_Leaf_Spot', 'Healthy']


In [4]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# === Load pretrained EfficientNet-B0 ===
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

# === Freeze all pretrained layers ===
for param in model.parameters():
    param.requires_grad = False

# === Replace the final classification layer ===
# EfficientNet's classifier is typically [Dropout, Linear]
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, 4)  # 4 classes: Blight, Common_Rust, Gray_Leaf_Spot, Healthy

# === Move model to GPU/CPU ===
model = model.to(device)

# === Check which layers are trainable ===
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Trainable layer: {name}")


Using device: cuda
Trainable layer: classifier.1.weight
Trainable layer: classifier.1.bias


In [5]:
import torch.optim as optim
import torch.nn as nn


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier[1].parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)


In [6]:
num_epochs = 50
best_acc=0.0
best_test=0.0
save_path = "/home/23ucs747/mini-project-sem5/models/efficientnet_b0_base.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
from tqdm import tqdm

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 30)
    
    # --- Training ---
    model.train()
    train_loss, correct, total = 0, 0, 0
    
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    train_acc = 100 * correct / total
    avg_train_loss = train_loss / len(train_loader)
    
    # --- Testing ---
    model.eval()
    test_loss, correct, total = 0, 0, 0
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    test_acc = 100 * correct / total
    avg_test_loss = test_loss / len(test_loader)
    if train_acc > best_acc :
        best_acc=train_acc
        best_test=test_acc
        torch.save(model.state_dict(), save_path)
        print(f"Retrained model saved at: {save_path}")
    
    print(f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Test Loss:  {avg_test_loss:.4f}, Test Acc:  {test_acc:.2f}%")
    
    scheduler.step()



Epoch 1/50
------------------------------


Train Loss: 0.5674, Train Acc: 80.50%
Test Loss:  0.2885, Test Acc:  90.00%

Epoch 2/50
------------------------------


Train Loss: 0.3530, Train Acc: 87.13%
Test Loss:  0.2453, Test Acc:  90.36%

Epoch 3/50
------------------------------


Train Loss: 0.3285, Train Acc: 87.16%
Test Loss:  0.2198, Test Acc:  91.79%

Epoch 4/50
------------------------------


Train Loss: 0.2999, Train Acc: 88.71%
Test Loss:  0.2065, Test Acc:  92.86%

Epoch 5/50
------------------------------


...

Train Loss: 0.2468, Train Acc: 90.44%
Test Loss:  0.1681, Test Acc:  94.88%

Epoch 47/50
------------------------------


Train Loss: 0.2414, Train Acc: 91.22%
Test Loss:  0.1657, Test Acc:  94.76%

Epoch 48/50
------------------------------


Train Loss: 0.2517, Train Acc: 90.35%
Test Loss:  0.1700, Test Acc:  93.81%

Epoch 49/50
------------------------------


Train Loss: 0.2605, Train Acc: 90.53%
Test Loss:  0.1724, Test Acc:  94.29%

Epoch 50/50
------------------------------


Train Loss: 0.2512, Train Acc: 90.56%
Test Loss:  0.1720, Test Acc:  93.81%
